In [1]:
import os
from typing import Optional

from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import SecretStr, field_validator

from generate_prompts import get_prompt

I'm trying to decide what workout to do tomorrow. today (Tuesday) I did bootcamp. yesterday (Monday) I did cycling. the day before (Sunday) I did megaformer pilates. my preferences: i prefer to be done with my workout and shower by 9:30am on weekdays, I don't want to wake up too early, i like to vary my exercises day to day, intense cardio (running & cycling) are better for my ADHD and anxiety symptoms, I prefer fitness classes over solo activities like running or swimming laps because i tend to procrastinate starting the latter. options: Yoga @ Sui Yoga · Soho SoHo at 8:00 AM 40 min away, Yoga @ Brooklyn Yoga Project Carroll Gardens at 7:30 AM 40 min away, Cycling @ SoulCycle · Williamsburg at 7:00 AM 50 min away, Yoga @ ID Hot Yoga - Lower East Side Lower East Side at 8:30 AM 40 min away, Gym Time @ Retro Fitness · One New York Plaza Financial District at 8:00 AM 30 min away, Gym Time @ Crunch Gym · Union Square Union Square at 7:00 AM 35 min away, lift weights @ home at ~8am 0m away

LOAD MODEL

In [2]:


from langchain_ollama import OllamaLLM

load_dotenv()
#model names
FREE_REKA_FLASH = "rekaai/reka-flash-3:free"
FREE_MOONSHOT = "moonshotai/kimi-vl-a3b-thinking:free"
FREE_MAVERICK = "meta-llama/llama-3.3-70b-instruct:free"


class ChatOpenRouter(ChatOpenAI):
    openai_api_base: str
    model_name: str

    def __init__(self,
                 model_name: str,
                 openai_api_key: Optional[str] = None,
                 openai_api_base: str = "https://openrouter.ai/api/v1",
                 **kwargs):
        openai_api_key = openai_api_key or SecretStr(os.getenv('OPENROUTER_API_KEY'))

        super().__init__(openai_api_base=openai_api_base, openai_api_key=openai_api_key, model_name=model_name,
                         **kwargs)


# llm = ChatOpenRouter(
#     model_name=FREE_REKA_FLASH, temperature=0.3
# )
llm = OllamaLLM(model="llama3.2", temperature=0.3)

In [3]:
from pydantic import BaseModel, field_validator, TypeAdapter
from typing import List, Dict
import re

valid_workout_list = ["cycling", "dance", "running"]

class WorkoutEntry(BaseModel):
    workout: str
    location: str
    time: str
    
    @field_validator('workout')
    def workout_must_be_valid(cls, v):
        if v.lower() not in valid_workout_list:
            raise ValueError(f"Workout must be one of: {', '.join(valid_workout_list)}")
        return v.lower()
    
    @field_validator('time')
    def time_must_be_valid_format(cls, v):
        if not re.match(r'^(1[0-2]|0?[1-9]):[0-5][0-9] (AM|PM)$', v):
            raise ValueError('Time must be in format like "8:30 AM" or "12:45 PM"')
        return v
workout_list_model = TypeAdapter(List[WorkoutEntry])

INVOKE

In [4]:
from generate_prompts import SLEEP, choice_descs, prev_workouts, VARIETY, EFFICIENT, FOCUS, STRUCTURE
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Step 0: Filter by sleep preference
sleep_preference_prompt = PromptTemplate.from_template(
    """Step 0: Consider these workout options:
```""" + choice_descs + """"```
    
List all options that are appropriate given the sleep preference: {SLEEP}"""
)

# Step 1: Filter by variety preference
variety_prompt = PromptTemplate.from_template(
    """Step 1: Given these options:
{sleep_preference_result}
    
And previous workouts:
""" + prev_workouts + """
    
succinctly list the subset of these options fits the variety preference: """ + VARIETY
)

# Step 2: Select top 1-3 options based on remaining preferences
top3_prompt = PromptTemplate.from_template(
    """Step 2: Given these options:
{variety_result}
    
List the top 1-3 options that best reflect these preferences:
- Efficiency: """ + EFFICIENT + """
- ADHD consideration: """ + FOCUS + """
- Structure preference: """ + STRUCTURE + """

give your answer as a json list of dicts with keys `workout, location, time`"""
)

sleep_chain = sleep_preference_prompt | llm | StrOutputParser()
variety_chain = variety_prompt | llm | StrOutputParser()
top3_chain = top3_prompt | llm | JsonOutputParser() 

chain = (
        RunnablePassthrough.assign(sleep_preference_result=sleep_chain) |
        RunnablePassthrough.assign(variety_result=variety_chain) |
        RunnablePassthrough.assign(top3_result=top3_chain)
)
# Invoke the chain with all inputs
result = chain.invoke({
    "SLEEP": SLEEP,
})
result["top3_result"]

I'm trying to decide what workout to do tomorrow. today (Tuesday) I did bootcamp. yesterday (Monday) I did cycling. the day before (Sunday) I did megaformer pilates. my preferences: intense cardio (running & cycling) are better for my ADHD and anxiety symptoms, I prefer fitness classes over solo activities like running or swimming laps because i tend to procrastinate starting the latter, i like to vary my exercises day to day, i prefer to be done with my workout and shower by 9:30am on weekdays, I don't want to wake up too early. options: Yoga @ Sui Yoga · Soho SoHo at 8:00 AM 40 min away, Yoga @ Brooklyn Yoga Project Carroll Gardens at 7:30 AM 40 min away, Cycling @ SoulCycle · Williamsburg at 8:00 AM 50 min away, peloton spin workout @ home at ~8am 0m away, Barre @ The Bar Method · Brooklyn - Williamsburg Williamsburg at 7:00 AM 50 min away, Yoga @ VERAYOGA (Tribeca) Tribeca at 7:50 AM 35 min away, Running @ Mile High Run Club · Noho NoHo at 7:30 AM 40 min away, Gym Time @ RS Strengt

[{'workout': "Running @ Barry's · Brooklyn Heights",
  'location': 'Brooklyn Heights',
  'time': '8:30 AM'},
 {'workout': 'Cycling @ CycleBar · NoHo',
  'location': 'NoHo',
  'time': '8:15 AM'},
 {'workout': 'Gym Time @ Crunch Gym · FiDi',
  'location': 'Financial District',
  'time': '8:00 AM'}]

In [5]:

from json_extractor import JsonExtractor
import pandas as pd

workout_list = result["top3_result"]
if workout_list:
    print(workout_list)
else:
    print(result["top3_result"])

df = pd.DataFrame(workout_list)

df

[{'workout': "Running @ Barry's · Brooklyn Heights", 'location': 'Brooklyn Heights', 'time': '8:30 AM'}, {'workout': 'Cycling @ CycleBar · NoHo', 'location': 'NoHo', 'time': '8:15 AM'}, {'workout': 'Gym Time @ Crunch Gym · FiDi', 'location': 'Financial District', 'time': '8:00 AM'}]


,workout,location,time
0,Running @ Barry's · Brooklyn Heights,Brooklyn Heights,8:30 AM
1,Cycling @ CycleBar · NoHo,NoHo,8:15 AM
2,Gym Time @ Crunch Gym · FiDi,Financial District,8:00 AM
